### Import pretrained model (Dectectron2) for object segementation.

In [23]:
import detectree as dtr
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio import plot
from skimage import measure
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import os 
from pathlib import Path


In [26]:
# Base directory path for the .tiff files
base_directory = "/Users/deep/Desktop/TreeCanopyClassification/TreeClassification_V2/stockton_image_tiles/"

# Get all .tiff files in the directory
tile_paths = list(Path(base_directory).glob("*.tiff"))

# List to store all polygons
all_polygons = []

# Loop through each .tiff image
for tile_path in tile_paths:
    # Predict tree/non-tree pixels
    y_pred = dtr.Classifier().predict_img(tile_path)
    
    # # Side-by-side plot of the tile and the predicted tree/non-tree pixels
    # figwidth, figheight = plt.rcParams["figure.figsize"]
    # fig, axes = plt.subplots(1, 2, figsize=(2 * figwidth, figheight))
    # with rio.open(tile_path) as src:
    #     plot.show(src, ax=axes[0])
    #     print(f"Raster width: {src.width}")
    #     print(f"Raster height: {src.height}")
    #     print(f"Raster count (number of bands): {src.count}")
    #     print(f"Raster crs (coordinate reference system): {src.crs}")
    #     print(f"Raster transform: {src.transform}")
    # axes[1].imshow(y_pred)
    # plt.show()

    # Read the .tiff image to get transform and CRS
    with rio.open(tile_path) as src:
        transform = src.transform
        crs = src.crs

    # Find contours
    contours = measure.find_contours(y_pred, 0.5)

    # Convert contours to polygons
    for contour in contours:
        # Only process contours with at least 4 points
        if len(contour) >= 4:
            # Flip the Y coordinates
            contour[:, 0] = y_pred.shape[0] - contour[:, 0]
            # Rotate the contour
            rotated_contour = np.array([contour[:, 1], y_pred.shape[0] - contour[:, 0]]).T
            # Create a polygon
            poly = Polygon([transform * (x, y) for x, y in rotated_contour])
            all_polygons.append(poly)

# Create a combined GeoDataFrame
gdf = gpd.GeoDataFrame({"geometry": all_polygons}, crs=crs)

# Save the combined GeoDataFrame as a single shapefile
shapefile_path = "/Users/deep/Desktop/shapefiles/combined_tree_polygons.shp"
gdf.to_file(shapefile_path)

print(f"Combined shapefile saved to: {shapefile_path}")

/Users/deep/miniconda3/envs/detectree/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.4.1.post1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/deep/miniconda3/envs/detectree/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.4.1.post1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/deep/miniconda3/envs/detectree/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEn

Combined shapefile saved to: /Users/deep/Desktop/shapefiles/combined_tree_polygons.shp


In [ ]:
tiff_path = "/Users/deep/Desktop/TreeCanopyClassification/TreeClassification_V2/stockton_image_tiles/tile_-121.36510000000001_37.96396666666664.tiff"
model_path = "/Users/deep/Desktop/TreeCanopyClassification/TreeClassification_V2/urban_trees_Cambridge_20230630.pth"